In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

In [2]:
raw_df = pd.read_csv('listings_details.csv')
raw_df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20191204162729,2019-12-07,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,1,1,0,0,0.39
1,3831,https://www.airbnb.com/rooms/3831,20191204162729,2019-12-07,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1,1,0,0,4.67
2,5099,https://www.airbnb.com/rooms/5099,20191204162729,2019-12-06,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,none,My neighborhood in Midtown East is called Murr...,...,f,f,moderate,t,t,1,1,0,0,0.60
3,5121,https://www.airbnb.com/rooms/5121,20191204162729,2019-12-06,BlissArtsSpace!,NaN,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,0.38
4,5178,https://www.airbnb.com/rooms/5178,20191204162729,2019-12-05,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"You will use one large, furnished, private roo...",Please don’t expect the luxury here just a bas...,none,"Theater district, many restaurants around here.",...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,3.52


In [3]:
key_list = list(raw_df.keys())
key_list.sort()
key_list

['access',
 'accommodates',
 'amenities',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bed_type',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'calendar_last_scraped',
 'calendar_updated',
 'cancellation_policy',
 'city',
 'cleaning_fee',
 'country',
 'country_code',
 'description',
 'experiences_offered',
 'extra_people',
 'first_review',
 'guests_included',
 'has_availability',
 'host_about',
 'host_acceptance_rate',
 'host_has_profile_pic',
 'host_id',
 'host_identity_verified',
 'host_is_superhost',
 'host_listings_count',
 'host_location',
 'host_name',
 'host_neighbourhood',
 'host_picture_url',
 'host_response_rate',
 'host_response_time',
 'host_since',
 'host_thumbnail_url',
 'host_total_listings_count',
 'host_url',
 'host_verifications',
 'house_rules',
 'id',
 'instan

In [4]:
raw_df.shape

(50599, 106)

In [5]:
#extract variables that are relevant to this project
df = raw_df[['room_type','cleaning_fee', 'host_name', 'name',
'neighbourhood_group_cleansed','neighbourhood_cleansed',
 'cancellation_policy','review_scores_rating', 'reviews_per_month',
 'minimum_nights', 'host_is_superhost', 'host_since', 
'property_type','price', 'maximum_nights', 'host_total_listings_count', 'host_name', 'latitude', 'longitude',
]]

In [6]:
df

,room_type,cleaning_fee,host_name,name,neighbourhood_group_cleansed,neighbourhood_cleansed,cancellation_policy,review_scores_rating,reviews_per_month,minimum_nights,host_is_superhost,host_since,property_type,price,maximum_nights,host_total_listings_count,host_name,latitude,longitude
0,Entire home/apt,$95.00,Jennifer,Skylit Midtown Castle,Manhattan,Midtown,strict_14_with_grace_period,94.0,0.39,10,f,2008-09-09,Apartment,$225.00,1125,6.0,Jennifer,40.75362,-73.98377
1,Entire home/apt,NaN,LisaRoxanne,Cozy Entire Floor of Brownstone,Brooklyn,Clinton Hill,moderate,90.0,4.67,1,f,2008-12-07,Guest suite,$89.00,730,1.0,LisaRoxanne,40.68514,-73.95976
2,Entire home/apt,$125.00,Chris,Large Cozy 1 BR Apartment In Midtown East,Manhattan,Murray Hill,moderate,90.0,0.60,3,f,2009-02-02,Apartment,$200.00,21,1.0,Chris,40.74767,-73.97500
3,Private room,$0.00,Garon,BlissArtsSpace!,Brooklyn,Bedford-Stuyvesant,strict_14_with_grace_period,90.0,0.38,29,f,2009-02-03,Apartment,$60.00,730,1.0,Garon,40.68688,-73.95596
4,Private room,$15.00,Shunichi,Large Furnished Room Near B'way,Manhattan,Hell's Kitchen,strict_14_with_grace_period,84.0,3.52,2,f,2009-03-03,Apartment,$79.00,14,1.0,Shunichi,40.76489,-73.98493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50594,Entire home/apt,NaN,Archi,"Happy, accommodating,confortable & súper CLEAN!",Bronx,Fordham,flexible,NaN,NaN,1,f,2019-12-03,Apartment,$200.00,28,2.0,Archi,40.86173,-73.89413
50595,Entire home/apt,$75.00,Hugo,Cozy Modern Cottage Apartment,Queens,Ditmars Steinway,moderate,NaN,NaN,3,f,2016-03-06,Apartment,$150.00,14,0.0,Hugo,40.77645,-73.90754
50596,Entire home/apt,NaN,Archi,"Súper clean, super confortable,súper ACCOMMODA...",Bronx,Fordham,flexible,NaN,NaN,1,f,2019-12-03,Apartment,$225.00,29,2.0,Archi,40.86239,-73.89468
50597,Private room,NaN,Joel And Julian,Captivating Room C,Brooklyn,Bedford-Stuyvesant,flexible,NaN,NaN,2,f,2018-10-08,House,$70.00,1125,2.0,Joel And Julian,40.69313,-73.95031


In [7]:
#make a df for Woodhaven neighborhood
woodhaven_df = df[df.neighbourhood_cleansed=='Woodhaven']
woodhaven_df

,room_type,cleaning_fee,host_name,name,neighbourhood_group_cleansed,neighbourhood_cleansed,cancellation_policy,review_scores_rating,reviews_per_month,minimum_nights,host_is_superhost,host_since,property_type,price,maximum_nights,host_total_listings_count,host_name,latitude,longitude
2778,Private room,$0.00,Antonio,Small Cozy Room Wifi & AC near JFK,Queens,Woodhaven,flexible,92.0,5.57,2,f,2013-10-07,House,$29.00,14,3.0,Antonio,40.68968,-73.85219
4065,Private room,$10.00,Yolanda,"Bright Room Near JFK ,LGA & Train with AC",Queens,Woodhaven,moderate,95.0,1.34,3,f,2014-05-02,House,$53.00,60,2.0,Yolanda,40.69550,-73.86281
4070,Private room,$10.00,Yolanda,"Bright Large Room near JFK, LGA & Train 2 beds",Queens,Woodhaven,strict_14_with_grace_period,90.0,2.18,3,f,2014-05-02,House,$40.00,60,2.0,Yolanda,40.69414,-73.86324
4380,Entire home/apt,$160.00,Rabah,Whole beautiful house with Backyard,Queens,Woodhaven,moderate,NaN,NaN,5,f,2012-04-06,House,$250.00,30,2.0,Rabah,40.69165,-73.84841
6197,Private room,$29.00,Troy,"Super Clean, Quiet & Spacious Room",Queens,Woodhaven,flexible,90.0,0.17,3,f,2014-12-08,Apartment,$31.00,31,1.0,Troy,40.69634,-73.85630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47016,Private room,$30.00,Penny,Walk 2min to supermarket food st and subway,Queens,Woodhaven,moderate,93.0,1.64,2,f,2017-02-27,Townhouse,$59.00,1125,13.0,Penny,40.69382,-73.86703
47803,Private room,NaN,Nicolas,1 Bedroom Suite.,Queens,Woodhaven,flexible,NaN,NaN,1,f,2019-09-07,Earth house,$87.00,15,0.0,Nicolas,40.69380,-73.86630
48049,Private room,$12.00,Jane,"Bright & Elegant, Convenient to Manhattan and ...",Queens,Woodhaven,moderate,NaN,NaN,1,t,2016-01-27,House,$59.00,99,3.0,Jane,40.68902,-73.85006
48316,Private room,NaN,Diane,Chic Cute Room,Queens,Woodhaven,flexible,NaN,NaN,1,f,2017-09-29,House,$60.00,7,1.0,Diane,40.68795,-73.85400


In [8]:
#get number of columns and rows
df.shape

(50599, 19)

In [9]:
df.isnull().sum()

room_type                           0
cleaning_fee                    10491
host_name                         563
name                               17
neighbourhood_group_cleansed        0
neighbourhood_cleansed              0
cancellation_policy                 0
review_scores_rating            11162
reviews_per_month               10220
minimum_nights                      0
host_is_superhost                 563
host_since                        563
property_type                       0
price                               0
maximum_nights                      0
host_total_listings_count         563
host_name                         563
latitude                            0
longitude                           0
dtype: int64

In [10]:
#fill na/null values with 0
df = df.fillna(0)

In [11]:
df.isnull().sum()

room_type                       0
cleaning_fee                    0
host_name                       0
name                            0
neighbourhood_group_cleansed    0
neighbourhood_cleansed          0
cancellation_policy             0
review_scores_rating            0
reviews_per_month               0
minimum_nights                  0
host_is_superhost               0
host_since                      0
property_type                   0
price                           0
maximum_nights                  0
host_total_listings_count       0
host_name                       0
latitude                        0
longitude                       0
dtype: int64

In [12]:
#checking the data type for each variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50599 entries, 0 to 50598
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   room_type                     50599 non-null  object 
 1   cleaning_fee                  50599 non-null  object 
 2   host_name                     50599 non-null  object 
 3   name                          50599 non-null  object 
 4   neighbourhood_group_cleansed  50599 non-null  object 
 5   neighbourhood_cleansed        50599 non-null  object 
 6   cancellation_policy           50599 non-null  object 
 7   review_scores_rating          50599 non-null  float64
 8   reviews_per_month             50599 non-null  float64
 9   minimum_nights                50599 non-null  int64  
 10  host_is_superhost             50599 non-null  object 
 11  host_since                    50599 non-null  object 
 12  property_type                 50599 non-null  object 
 13  p

In [13]:
#look at data types in this column
df['cleaning_fee'].apply(type)

0        <class 'str'>
1        <class 'int'>
2        <class 'str'>
3        <class 'str'>
4        <class 'str'>
             ...      
50594    <class 'int'>
50595    <class 'str'>
50596    <class 'int'>
50597    <class 'int'>
50598    <class 'int'>
Name: cleaning_fee, Length: 50599, dtype: object

In [14]:
#check the types of data in a column using value_counts()
df['cleaning_fee'].apply(type).value_counts()

<class 'str'>    40108
<class 'int'>    10491
Name: cleaning_fee, dtype: int64

In [15]:
#change the dtype of those numeric values from object to float64
#will get error trying to use string fucntions on an integer eg. using str.replace on integers
#df['cleaning_fee'] contains both str and int
# df['cleaning_fee'] = df['cleaning_fee'].str.replace('$','')


In [16]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$','').replace(',',''))
    return(x)

In [17]:
#check the function
num = clean_currency('$44.55')
num

'44.55'

In [18]:
df['cleaning_fee'] = df['cleaning_fee'].apply(clean_currency).astype('float')
df['cleaning_fee']

0         95.0
1          0.0
2        125.0
3          0.0
4         15.0
         ...  
50594      0.0
50595     75.0
50596      0.0
50597      0.0
50598      0.0
Name: cleaning_fee, Length: 50599, dtype: float64

In [19]:
#check data types again to see that class 'int' are all now 'float'
df['cleaning_fee'].apply(type).value_counts()

<class 'float'>    50599
Name: cleaning_fee, dtype: int64

In [20]:
df['price'] = df['price'].apply(clean_currency).astype('float')
df['price']

0        225.0
1         89.0
2        200.0
3         60.0
4         79.0
         ...  
50594    200.0
50595    150.0
50596    225.0
50597     70.0
50598    100.0
Name: price, Length: 50599, dtype: float64

In [21]:
df['room_type'].apply(type).value_counts()

<class 'str'>    50599
Name: room_type, dtype: int64

In [22]:
df['host_since'].apply(type).value_counts()

<class 'str'>    50036
<class 'int'>      563
Name: host_since, dtype: int64

In [23]:
#convert host_since from object to datetime64
pd.to_datetime(df.host_since)

0       2008-09-09
1       2008-12-07
2       2009-02-02
3       2009-02-03
4       2009-03-03
           ...    
50594   2019-12-03
50595   2016-03-06
50596   2019-12-03
50597   2018-10-08
50598   2015-02-18
Name: host_since, Length: 50599, dtype: datetime64[ns]

In [24]:
def convert_datetime(x):
    """ convert to datetime"""
    return(pd.to_datetime(x))

In [25]:
df['host_since'] = df['host_since'].apply(convert_datetime)
df['host_since']

0       2008-09-09
1       2008-12-07
2       2009-02-02
3       2009-02-03
4       2009-03-03
           ...    
50594   2019-12-03
50595   2016-03-06
50596   2019-12-03
50597   2018-10-08
50598   2015-02-18
Name: host_since, Length: 50599, dtype: datetime64[ns]

In [26]:
#check datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50599 entries, 0 to 50598
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   room_type                     50599 non-null  object        
 1   cleaning_fee                  50599 non-null  float64       
 2   host_name                     50599 non-null  object        
 3   name                          50599 non-null  object        
 4   neighbourhood_group_cleansed  50599 non-null  object        
 5   neighbourhood_cleansed        50599 non-null  object        
 6   cancellation_policy           50599 non-null  object        
 7   review_scores_rating          50599 non-null  float64       
 8   reviews_per_month             50599 non-null  float64       
 9   minimum_nights                50599 non-null  int64         
 10  host_is_superhost             50599 non-null  object        
 11  host_since                  

In [27]:
df['host_since'].apply(type).value_counts()

<class 'pandas._libs.tslibs.timestamps.Timestamp'>    50599
Name: host_since, dtype: int64

In [28]:
queens_df = df[df.neighbourhood_group_cleansed == 'Queens']

In [42]:
queens_df

,room_type,cleaning_fee,host_name,name,neighbourhood_group_cleansed,neighbourhood_cleansed,cancellation_policy,review_scores_rating,reviews_per_month,minimum_nights,host_is_superhost,host_since,property_type,price,maximum_nights,host_total_listings_count,host_name,latitude,longitude
38,Private room,0.0,0,"1 Stop fr. Manhattan! Private Suite,Landmark B...",Queens,Long Island City,strict_14_with_grace_period,98.0,2.33,2,0,1970-01-01,Townhouse,130.0,20,0.0,0,40.74771,-73.94740
70,Private room,0.0,Justin,King of Queens w/ Private Bathroom,Queens,Woodside,strict_14_with_grace_period,99.0,0.17,7,f,2010-01-12,Condominium,90.0,365,1.0,Justin,40.74590,-73.90764
127,Private room,0.0,Fredah,Fully Furnished Basement Apartment,Queens,Flushing,flexible,80.0,0.01,2,f,2010-06-07,House,140.0,730,1.0,Fredah,40.74028,-73.83168
146,Private room,0.0,Myung,"A room w/ a Manhattan view, 12/23/20 longer stay",Queens,Sunnyside,strict_14_with_grace_period,98.0,0.25,30,f,2010-04-18,Apartment,79.0,180,1.0,Myung,40.74559,-73.92313
167,Entire home/apt,200.0,Mark,Beautiful Queens Brownstone! - 5BR,Queens,Ridgewood,strict_14_with_grace_period,88.0,0.13,4,f,2010-08-17,Townhouse,350.0,730,5.0,Mark,40.70382,-73.89797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50540,Private room,0.0,Malcolm/Carolyn,Beach 28th St 3,Queens,Far Rockaway,flexible,0.0,0.00,1,f,2012-04-11,Apartment,78.0,1125,3.0,Malcolm/Carolyn,40.59884,-73.76293
50541,Private room,0.0,Malcolm/Carolyn,Beach 28th Surfers delight 2,Queens,Far Rockaway,flexible,0.0,0.00,1,f,2012-04-11,Apartment,78.0,1125,3.0,Malcolm/Carolyn,40.59803,-73.76080
50573,Private room,25.0,Anais,"Hidden Spot, Amazing room with two Queens",Queens,Long Island City,strict_14_with_grace_period,0.0,0.00,1,f,2016-06-05,Villa,65.0,30,3.0,Anais,40.76017,-73.93024
50586,Private room,20.0,Dave,Astoria! 15 minutes away from Central Park/Tim...,Queens,Astoria,moderate,0.0,0.00,3,f,2015-07-21,Apartment,75.0,5,0.0,Dave,40.75721,-73.91574


In [29]:
brooklyn_df = df[df.neighbourhood_group_cleansed == 'Brooklyn']

In [43]:
brooklyn_df

,room_type,cleaning_fee,host_name,name,neighbourhood_group_cleansed,neighbourhood_cleansed,cancellation_policy,review_scores_rating,reviews_per_month,minimum_nights,host_is_superhost,host_since,property_type,price,maximum_nights,host_total_listings_count,host_name,latitude,longitude
1,Entire home/apt,0.0,LisaRoxanne,Cozy Entire Floor of Brownstone,Brooklyn,Clinton Hill,moderate,90.0,4.67,1,f,2008-12-07,Guest suite,89.0,730,1.0,LisaRoxanne,40.68514,-73.95976
3,Private room,0.0,Garon,BlissArtsSpace!,Brooklyn,Bedford-Stuyvesant,strict_14_with_grace_period,90.0,0.38,29,f,2009-02-03,Apartment,60.0,730,1.0,Garon,40.68688,-73.95596
8,Private room,67.0,Laurie,"Lovely Room 1, Garden, Best Area, Legal rental",Brooklyn,South Slope,strict_14_with_grace_period,94.0,1.35,4,t,2009-03-10,Townhouse,89.0,14,3.0,Laurie,40.66829,-73.98779
10,Entire home/apt,90.0,Allen & Irina,Only 2 stops to Manhattan studio,Brooklyn,Williamsburg,strict_14_with_grace_period,91.0,1.33,2,f,2009-05-06,Apartment,140.0,90,1.0,Allen & Irina,40.70837,-73.95352
11,Entire home/apt,30.0,Jane,Perfect for Your Parents + Garden,Brooklyn,Fort Greene,flexible,97.0,1.79,1,t,2009-05-17,Apartment,199.0,30,1.0,Jane,40.69169,-73.97185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50572,Private room,65.0,Micah,Beautiful Private Bedroom with Balcony,Brooklyn,Bedford-Stuyvesant,moderate,0.0,0.00,4,f,2016-11-30,Apartment,65.0,14,3.0,Micah,40.68967,-73.92985
50581,Entire home/apt,0.0,Enzo,Sunny & Baby-friendly South Prospect Park Condo,Brooklyn,Flatbush,moderate,0.0,0.00,6,f,2013-08-17,Apartment,155.0,1125,0.0,Enzo,40.65025,-73.96200
50587,Private room,0.0,Zain,Huge bedroom on top floor of Brooklyn brownstone,Brooklyn,Bedford-Stuyvesant,flexible,0.0,0.00,31,f,2015-07-01,House,45.0,31,1.0,Zain,40.68590,-73.95570
50597,Private room,0.0,Joel And Julian,Captivating Room C,Brooklyn,Bedford-Stuyvesant,flexible,0.0,0.00,2,f,2018-10-08,House,70.0,1125,2.0,Joel And Julian,40.69313,-73.95031


In [44]:
manhattan_df = df[df.neighbourhood_group_cleansed == 'Manhattan']

,room_type,cleaning_fee,host_name,name,neighbourhood_group_cleansed,neighbourhood_cleansed,cancellation_policy,review_scores_rating,reviews_per_month,minimum_nights,host_is_superhost,host_since,property_type,price,maximum_nights,host_total_listings_count,host_name,latitude,longitude
0,Entire home/apt,95.0,Jennifer,Skylit Midtown Castle,Manhattan,Midtown,strict_14_with_grace_period,94.0,0.39,10,f,2008-09-09,Apartment,225.0,1125,6.0,Jennifer,40.75362,-73.98377
2,Entire home/apt,125.0,Chris,Large Cozy 1 BR Apartment In Midtown East,Manhattan,Murray Hill,moderate,90.0,0.60,3,f,2009-02-02,Apartment,200.0,21,1.0,Chris,40.74767,-73.97500
4,Private room,15.0,Shunichi,Large Furnished Room Near B'way,Manhattan,Hell's Kitchen,strict_14_with_grace_period,84.0,3.52,2,f,2009-03-03,Apartment,79.0,14,1.0,Shunichi,40.76489,-73.98493
5,Private room,0.0,MaryEllen,Cozy Clean Guest Room - Family Apt,Manhattan,Upper West Side,flexible,98.0,0.95,2,f,2009-02-05,Apartment,79.0,14,1.0,MaryEllen,40.80178,-73.96723
6,Entire home/apt,40.0,Ben,Cute & Cozy Lower East Side 1 bdrm,Manhattan,Chinatown,strict_14_with_grace_period,94.0,1.29,1,t,2009-02-07,Apartment,150.0,1125,4.0,Ben,40.71344,-73.99037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50589,Entire home/apt,30.0,John,"Close to Everything, Very Quiet and Cozy Free ...",Manhattan,Murray Hill,moderate,0.0,0.00,4,f,2014-01-12,Apartment,190.0,9,0.0,John,40.74846,-73.97834
50590,Private room,0.0,Brigitte,Lovely Large Private Bedroom - Lots of Sunlight!,Manhattan,Harlem,flexible,0.0,0.00,2,f,2014-10-04,Apartment,40.0,1125,1.0,Brigitte,40.82908,-73.94756
50591,Private room,15.0,Jini,[E]Enjoy your NYC with PRIME location,Manhattan,Chelsea,moderate,0.0,0.00,2,f,2019-09-24,Apartment,70.0,1125,2.0,Jini,40.74741,-73.98992
50592,Entire home/apt,120.0,Emmanouil,Gorgeous Loft in the heart of Flatiron / Chelsea,Manhattan,Chelsea,flexible,0.0,0.00,30,f,2019-12-03,Apartment,300.0,1125,0.0,Emmanouil,40.74227,-73.99341


In [45]:
manhattan_df

,room_type,cleaning_fee,host_name,name,neighbourhood_group_cleansed,neighbourhood_cleansed,cancellation_policy,review_scores_rating,reviews_per_month,minimum_nights,host_is_superhost,host_since,property_type,price,maximum_nights,host_total_listings_count,host_name,latitude,longitude
0,Entire home/apt,95.0,Jennifer,Skylit Midtown Castle,Manhattan,Midtown,strict_14_with_grace_period,94.0,0.39,10,f,2008-09-09,Apartment,225.0,1125,6.0,Jennifer,40.75362,-73.98377
2,Entire home/apt,125.0,Chris,Large Cozy 1 BR Apartment In Midtown East,Manhattan,Murray Hill,moderate,90.0,0.60,3,f,2009-02-02,Apartment,200.0,21,1.0,Chris,40.74767,-73.97500
4,Private room,15.0,Shunichi,Large Furnished Room Near B'way,Manhattan,Hell's Kitchen,strict_14_with_grace_period,84.0,3.52,2,f,2009-03-03,Apartment,79.0,14,1.0,Shunichi,40.76489,-73.98493
5,Private room,0.0,MaryEllen,Cozy Clean Guest Room - Family Apt,Manhattan,Upper West Side,flexible,98.0,0.95,2,f,2009-02-05,Apartment,79.0,14,1.0,MaryEllen,40.80178,-73.96723
6,Entire home/apt,40.0,Ben,Cute & Cozy Lower East Side 1 bdrm,Manhattan,Chinatown,strict_14_with_grace_period,94.0,1.29,1,t,2009-02-07,Apartment,150.0,1125,4.0,Ben,40.71344,-73.99037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50589,Entire home/apt,30.0,John,"Close to Everything, Very Quiet and Cozy Free ...",Manhattan,Murray Hill,moderate,0.0,0.00,4,f,2014-01-12,Apartment,190.0,9,0.0,John,40.74846,-73.97834
50590,Private room,0.0,Brigitte,Lovely Large Private Bedroom - Lots of Sunlight!,Manhattan,Harlem,flexible,0.0,0.00,2,f,2014-10-04,Apartment,40.0,1125,1.0,Brigitte,40.82908,-73.94756
50591,Private room,15.0,Jini,[E]Enjoy your NYC with PRIME location,Manhattan,Chelsea,moderate,0.0,0.00,2,f,2019-09-24,Apartment,70.0,1125,2.0,Jini,40.74741,-73.98992
50592,Entire home/apt,120.0,Emmanouil,Gorgeous Loft in the heart of Flatiron / Chelsea,Manhattan,Chelsea,flexible,0.0,0.00,30,f,2019-12-03,Apartment,300.0,1125,0.0,Emmanouil,40.74227,-73.99341


In [46]:
df.groupby(by="neighbourhood_group_cleansed").mean()

,cleaning_fee,review_scores_rating,reviews_per_month,minimum_nights,price,maximum_nights,host_total_listings_count,latitude,longitude
neighbourhood_group_cleansed,,,,,,,,,
Bronx,35.924280,74.309465,1.527432,4.638683,86.948148,474.867490,2.548148,40.848570,-73.883476
Brooklyn,47.534330,75.907281,1.088097,6.424301,124.384619,2491.874204,2.645516,40.684405,-73.950756
Manhattan,64.666289,69.620526,0.970207,9.201450,210.947621,97973.594155,35.375215,40.765250,-73.974515
Queens,35.040026,75.794383,1.660491,6.061975,100.476598,546.102808,4.927534,40.730773,-73.869984
Staten Island,45.246032,79.653439,1.702328,4.219577,105.259259,420.317460,2.211640,40.611531,-74.104539
